# Music Genre Classification

## Recurrent Neural Network Implementation

Fabrizio Niro - Jacopo Signò

GTZAN Dataset - Music Genre Classification 

https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification

In [38]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
import pickle
import numpy as np
import torchvision
from d2l import torch as d2l
from torch.utils import data

In [39]:
with open('Processed_data/RNN/data_train', 'rb') as f:
    data_train = pickle.load(f)

with open('Processed_data/RNN/data_test', 'rb') as f:
    data_test = pickle.load(f)

In [40]:
print(len(data_train), len(data_train[0]))

8000 2


In [41]:
print(len(data_test), len(data_test[0]))

2000 2


In [42]:
batch_size = 64

train_iter = data.DataLoader(data_train, batch_size, shuffle=True,
                             num_workers = 2)

In [43]:
train_iter

In [44]:
next(iter(train_iter))

[tensor([[[-3.8980e+00, -3.7337e+00, -3.7626e+00,  ..., -2.1406e+00,
           -7.5624e-01, -2.8286e+00],
          [ 1.6081e+00,  1.7513e+00,  1.6185e+00,  ...,  3.2593e+00,
            3.5006e+00,  2.7701e+00],
          [ 4.9556e-01,  1.7857e-01, -3.4299e-02,  ...,  5.3055e-02,
            4.4444e-01,  1.0718e+00],
          ...,
          [ 5.5450e-01,  7.4449e-01,  7.7473e-01,  ..., -1.9278e-01,
           -3.4208e-01,  1.9200e-02],
          [ 1.5422e-01,  2.2074e-01,  2.5529e-01,  ..., -2.1722e-01,
           -6.8007e-01, -3.9191e-01],
          [ 7.4397e-01,  9.6589e-01,  9.7954e-01,  ...,  5.3046e-03,
           -4.7308e-01, -5.3522e-01]],
 
         [[-3.9725e+00, -3.9383e+00, -4.0158e+00,  ..., -2.2626e+00,
           -3.2860e+00, -3.7031e+00],
          [ 1.9031e+00,  1.9770e+00,  1.8553e+00,  ...,  3.1578e+00,
            2.7284e+00,  2.3419e+00],
          [-2.7974e-01, -2.7592e-01, -8.3819e-02,  ..., -9.7017e-01,
           -4.3939e-01, -2.0896e-01],
          ...,
    

In [45]:
test_iter = data.DataLoader(data_test, batch_size, shuffle=False,
                             num_workers = 2)

In [46]:
test_iter

In [47]:
next(iter(test_iter))[0].shape

torch.Size([64, 20, 129])

In [48]:
next(iter(test_iter))[0][0].shape

torch.Size([20, 129])

In [58]:
input_dim = data_train[0][0].shape[0] * data_train[0][0].shape[1]
input_dim

2580

In [206]:
classes_num = 10
classes_num

10

In [212]:
class ModelRNN(nn.Module):
    def __init__(self):
        super(ModelRNN, self).__init__()

        self.flat = nn.Flatten()
        self.rnn = nn.LSTM(input_dim, input_dim)
        
        self.body = nn.Sequential(
          nn.Linear(input_dim, 1000),
          nn.ReLU(),
          nn.Linear(1000, 200),
          nn.ReLU(),
          nn.Linear(200, classes_num),
          nn.Softmax()
          ) 

    def forward(self, x):

        x = self.flat(x)
        x, _ = self.rnn(x) # <- ignore second output
        x = self.body(x)

        return x

In [211]:
net = ModelRNN()

In [203]:
def init_weights(m): # argument m shall be a layer
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights); # apply() will call init_weights() providing it with each layer of net as actual argument m

In [204]:
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.001)

In [115]:
def train_epoch(net, train_iter, loss, updater):
    """The training loop"""
    # Set the model to training mode

    if isinstance(net, torch.nn.Module):
        net.train()
    # Sum of training loss, sum of training accuracy, nb. of examples
    metric = d2l.Accumulator(3)

    for X, y in train_iter:

        # Compute gradients and update parameters
        y_hat = net(X)
        l = loss(y_hat, y)
        updater.zero_grad()
        l.backward()
        updater.step()
        metric.add(float(l) * len(y), d2l.accuracy(y_hat, y), y.size().numel())
        
    # Return training loss and training accuracy
    return metric[0] / metric[2], metric[1] / metric[2]

In [116]:
def train(net, train_iter, test_iter, loss, num_epochs, updater):
    """Train a model"""
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
                        
    for epoch in range(num_epochs):
        
        train_metrics = train_epoch(net, train_iter, loss, updater)
        # ! WARNING !: d2l.evaluate_accuracy() no longer defined. Use d2l.evaluate_accuracy_gpu() instead
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))

    train_loss, train_acc = train_metrics
    return train_loss, train_acc

In [ ]:
num_epochs = 20
train_loss, train_acc = train(net, train_iter, test_iter, loss, num_epochs, trainer)
print("Loss: ", train_loss)
print("Accuracy: ", train_acc)